In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error
import seaborn as sns

In [3]:
df = pd.read_csv("employee_salaries.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4807 entries, 0 to 4806
Data columns (total 8 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Department                        4807 non-null   object 
 1   Division                          4807 non-null   object 
 2   Position Title                    4807 non-null   object 
 3   Employee Status                   4807 non-null   object 
 4   Initial Hire Date                 4807 non-null   object 
 5   Date in Position                  4807 non-null   object 
 6   Fair Labor Standards Act (FLSA)   4807 non-null   object 
 7   Base Salary                       4807 non-null   float64
dtypes: float64(1), object(7)
memory usage: 300.6+ KB


In [4]:
df.describe()

,Base Salary
count,4807.000000
mean,50301.175690
std,35402.525778
min,15.000000
25%,39575.000000
50%,50156.930000
75%,68014.000000
max,295130.000000


In [5]:
df.head(10)

,Department,Division,Position Title,Employee Status,Initial Hire Date,Date in Position,Fair Labor Standards Act (FLSA),Base Salary
0,City Council,City Council,Member Of Council,Elected Official,07/01/2014,07/01/2014,Exempt,25000.08
1,City Council,City Council,Member Of Council,Elected Official,07/01/2016,07/01/2016,Exempt,25000.00
2,City Council,City Council,Member Of Council,Elected Official,06/21/1999,08/31/2016,Exempt,25000.00
3,City Council,City Council,President Of Council,Elected Official,07/01/2016,07/01/2016,Exempt,27000.00
4,City Council,City Council,Member Of Council,Elected Official,01/01/2023,01/01/2023,Exempt,25000.00
5,City Council,City Council,Member Of Council,Elected Official,01/14/2021,01/14/2021,Exempt,25000.00
6,City Council,City Council,Member Of Council,Elected Official,07/24/1998,07/01/2010,Exempt,25000.00
7,City Council,City Council,Member Of Council,Elected Official,07/01/2018,07/01/2018,Exempt,25000.00
8,City Manager,Executive City Manager,City Manager,Appointed by City Council,07/26/2021,06/28/2023,Exempt,295130.00
9,City Manager,Executive City Manager,"Asst to the Cty Manager, Sr",Permanent Full-time,08/04/1998,05/04/2015,Exempt,120751.00


In [7]:
df.isnull().sum()

Department                          0
Division                            0
Position Title                      0
Employee Status                     0
Initial Hire Date                   0
Date in Position                    0
Fair Labor Standards Act (FLSA)     0
Base Salary                         0
dtype: int64

In [10]:
for col in df.columns:
    if df[col].isnull().sum() > 0 and df.col.dtype() in ("float", "int"):
        df[col].fillna(df[col].median(), inplace= True)
    elif df[col].isnull().sum() > 0 and  df.col.dtype() not in ("float", "int"):
        df[col].fillna("Missing", inplace=True)

In [14]:
df["Position Title"].value_counts()

Firefighter EMT - Advanced        240
Master Police Officer             235
Police Officer                    185
Youth Services Worker             135
Deputy Sheriff                    126
                                 ... 
Compliance Inspector                1
Custodian                           1
President Of Council                1
Health & Fitness Facilitator        1
Victim / Witness Prog Director      1
Name: Position Title, Length: 644, dtype: int64

In [15]:
df["Division"].value_counts()

Fire-Rescue                       482
Judicial - Sheriff and Jail       396
Hum Svc Div of Social Services    350
Police Field Operations-Sworn     325
RPOS-Youth Development            160
                                 ... 
NCSB - Forensic Services            1
NCSB-Youth Activity Admin           1
SABG Prevention                     1
NCSB STEP-VA FY 2023                1
ARPA-VSGP                           1
Name: Division, Length: 169, dtype: int64

In [17]:

df["Initial Hire Date"] = pd.to_datetime(df["Initial Hire Date"])
df["lifetime"] = (pd.Timestamp.today() - df["Initial Hire Date"]).dt.days
df

,Department,Division,Position Title,Employee Status,Initial Hire Date,Date in Position,Fair Labor Standards Act (FLSA),Base Salary,lifetime
0,City Council,City Council,Member Of Council,Elected Official,2014-07-01,07/01/2014,Exempt,25000.08,3653
1,City Council,City Council,Member Of Council,Elected Official,2016-07-01,07/01/2016,Exempt,25000.00,2922
2,City Council,City Council,Member Of Council,Elected Official,1999-06-21,08/31/2016,Exempt,25000.00,9142
3,City Council,City Council,President Of Council,Elected Official,2016-07-01,07/01/2016,Exempt,27000.00,2922
4,City Council,City Council,Member Of Council,Elected Official,2023-01-01,01/01/2023,Exempt,25000.00,547
...,...,...,...,...,...,...,...,...,...
4802,Commonwealths Attorney,Commonwealth's Attorney,Asst Commonwealth's Att II,Permanent Full-time,2022-08-22,08/22/2022,Exempt,91171.24,679
4803,Commonwealths Attorney,Commonwealth's Attorney,Executive Secy/Assistant CWA,State Comp Board Retirement,2000-10-04,01/04/2020,Nonexempt,29.52,8671
4804,Commonwealths Attorney,Commonwealth's Attorney,Legal Administrator CWA,State Comp Board Retirement,2021-06-01,01/01/2022,Exempt,84453.22,1126
4805,Commonwealths Attorney,Commonwealth's Attorney,Paralegal CWA,Permanent Full-time,2005-08-22,09/14/2019,Nonexempt,56458.75,6888


In [21]:
df = df[df["Base Salary"] > 1000]

In [23]:
df = df.reset_index()


In [25]:
df = df.drop(["index", "Initial Hire Date", "Date in Position"], axis=1)

In [26]:
df["Division"].value_counts()

Fire-Rescue                       481
Hum Svc Div of Social Services    341
Police Field Operations-Sworn     321
Police Investigative Svc-Sworn    118
RPOS - Landscape Services         111
                                 ... 
NCSB-Youth Services                 1
Transit - Directors Office          1
NCSB-Adult & Family Services        1
NCSB-ACR                            1
RyanWhite Quality Management        1
Name: Division, Length: 156, dtype: int64

In [27]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for col in df.select_dtypes("object"):
    df[col] = le.fit_transform(df[col])

df

,Department,Division,Position Title,Employee Status,Fair Labor Standards Act (FLSA),Base Salary,lifetime
0,6,8,373,1,0,25000.08,3653
1,6,8,373,1,0,25000.00,2922
2,6,8,373,1,0,25000.00,9142
3,6,8,436,1,0,27000.00,2922
4,6,8,373,1,0,25000.00,547
...,...,...,...,...,...,...,...
3828,13,152,567,6,0,48475.88,1841
3829,13,152,566,6,0,71000.00,3750
3830,13,18,42,2,0,91171.24,679
3831,13,18,340,8,0,84453.22,1126


In [28]:
df.describe()

,Department,Division,Position Title,Employee Status,Fair Labor Standards Act (FLSA),Base Salary,lifetime
count,3833.000000,3833.000000,3833.000000,3833.000000,3833.000000,3833.000000,3833.000000
mean,28.154448,76.285938,305.539003,2.211323,0.694234,63077.188312,4037.426298
std,9.482989,44.903226,155.537369,1.054223,0.460791,27678.903808,3655.717507
min,0.000000,0.000000,0.000000,0.000000,0.000000,5319.000000,14.000000
25%,21.000000,36.000000,204.000000,2.000000,0.000000,45013.000000,812.000000
50%,30.000000,65.000000,320.000000,2.000000,1.000000,55392.730000,2926.000000
75%,34.000000,117.000000,432.000000,2.000000,1.000000,74610.000000,6614.000000
max,43.000000,155.000000,586.000000,8.000000,1.000000,295130.000000,19256.000000


In [33]:
Y = df["Base Salary"]
X = df.drop(["Base Salary"], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, shuffle=True)

In [34]:
from sklearn.ensemble import RandomForestRegressor



params = {
    #numero de trees
    "n_estimators" : [170, 200, 240],
    #max depth of trees
    "max_depth": [None,10, 20]
    
}

grid = GridSearchCV(RandomForestRegressor(), param_grid=params, scoring="neg_mean_squared_error", cv=6, verbose=2)

grid.fit(X_train, y_train)
print(grid.best_params_)
print(grid.best_score_)

Fitting 6 folds for each of 16 candidates, totalling 96 fits
[CV] END ....................max_depth=None, n_estimators=50; total time=   0.2s
[CV] END ....................max_depth=None, n_estimators=50; total time=   0.2s
[CV] END ....................max_depth=None, n_estimators=50; total time=   0.2s
[CV] END ....................max_depth=None, n_estimators=50; total time=   0.2s
[CV] END ....................max_depth=None, n_estimators=50; total time=   0.2s
[CV] END ....................max_depth=None, n_estimators=50; total time=   0.2s
[CV] END ...................max_depth=None, n_estimators=100; total time=   0.4s
[CV] END ...................max_depth=None, n_estimators=100; total time=   0.5s
[CV] END ...................max_depth=None, n_estimators=100; total time=   0.4s
[CV] END ...................max_depth=None, n_estimators=100; total time=   0.4s
[CV] END ...................max_depth=None, n_estimators=100; total time=   0.5s
[CV] END ...................max_depth=None, n_es

In [35]:
y_pred = grid.best_estimator_.predict(X_test)
mean_absolute_error(y_test, y_pred)

6563.668506632178

In [36]:
from sklearn.ensemble import GradientBoostingRegressor
GradientBoostingRegressor()

params = {
    #3*3*3 combinaciones
    "learning_rate": [0.05, 0.1, 0.3],
    "n_estimators" : [180, 200, 220],
    "max_depth": [3,5,7]
    
}
grid = GridSearchCV(GradientBoostingRegressor(), param_grid=params, scoring="neg_mean_squared_error", verbose=2)
grid.fit(X_train, y_train)

Fitting 5 folds for each of 27 candidates, totalling 135 fits
[CV] END ..learning_rate=0.05, max_depth=3, n_estimators=180; total time=   0.2s
[CV] END ..learning_rate=0.05, max_depth=3, n_estimators=180; total time=   0.2s
[CV] END ..learning_rate=0.05, max_depth=3, n_estimators=180; total time=   0.2s
[CV] END ..learning_rate=0.05, max_depth=3, n_estimators=180; total time=   0.3s
[CV] END ..learning_rate=0.05, max_depth=3, n_estimators=180; total time=   0.2s
[CV] END ..learning_rate=0.05, max_depth=3, n_estimators=200; total time=   0.2s
[CV] END ..learning_rate=0.05, max_depth=3, n_estimators=200; total time=   0.2s
[CV] END ..learning_rate=0.05, max_depth=3, n_estimators=200; total time=   0.2s
[CV] END ..learning_rate=0.05, max_depth=3, n_estimators=200; total time=   0.2s
[CV] END ..learning_rate=0.05, max_depth=3, n_estimators=200; total time=   0.3s
[CV] END ..learning_rate=0.05, max_depth=3, n_estimators=220; total time=   0.3s
[CV] END ..learning_rate=0.05, max_depth=3, n_e

GridSearchCV(estimator=GradientBoostingRegressor(),
             param_grid={'learning_rate': [0.05, 0.1, 0.3],
                         'max_depth': [3, 5, 7],
                         'n_estimators': [180, 200, 220]},
             scoring='neg_mean_squared_error', verbose=2)

In [37]:
grid.best_params_

{'learning_rate': 0.3, 'max_depth': 3, 'n_estimators': 220}

In [38]:
grid.best_estimator_

GradientBoostingRegressor(learning_rate=0.3, n_estimators=220)

In [39]:
y_pred = grid.best_estimator_.predict(X_test)
mean_absolute_error(y_test, y_pred)

6794.415188141955

In [40]:
mean_absolute_percentage_error(y_test, y_pred)

0.09684990535887265